<a href="https://colab.research.google.com/github/younesabdolmalaky/Pistachio/blob/main/ResNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.applications.vgg16 import VGG16
import os
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report
import numpy as np
from keras import metrics
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pathlib
import random
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Conv2D, MaxPool2D, MaxPooling2D
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from keras import applications
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2L
from tensorflow.keras.applications.resnet50 import ResNet50


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
img_size = 512

In [5]:
base_model = ResNet50(weights='imagenet', include_top=False,
                            input_shape=(600,600,3))
# freeze extraction layers
base_model.trainable = False

# add custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
x = Dense(4096,activation="relu")(x)
x = Dense(4096,activation="relu")(x)
x = Dropout(0.2)(x)
x = Dense(2096,activation="relu")(x)
predictions = Dense(2, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# confirm unfrozen layers
for layer in model.layers:
    if layer.trainable==True:
        print(layer)

94765736/94765736 [==============================] - 5s 0us/step


In [6]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 600, 600, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 606, 606, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 300, 300, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [7]:
callbacks = [EarlyStopping(monitor='val_loss', patience=5, verbose=1),
                ModelCheckpoint('model.hdf5',
                                 save_best_only=True)]

In [8]:
opt = Adam(learning_rate=0.005)
model.compile(
  loss='categorical_crossentropy',
  optimizer=opt,
  metrics=['accuracy', metrics.Precision(), metrics.Recall()]
)

In [9]:
image_generator = ImageDataGenerator(rescale = 1.0/255.0,
                                   rotation_range = 0.5,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.1,
                                   horizontal_flip = True,
                                   fill_mode = 'nearest'
                                  )

image_generator_test = ImageDataGenerator(rescale = 1.0/255.0)


In [10]:
train_data = image_generator.flow_from_directory(
    '/content/drive/MyDrive/Pistachio/train',
    target_size=(512, 512),
    color_mode="rgb",
    class_mode='categorical',
    batch_size=32
)

Found 1717 images belonging to 2 classes.


In [11]:
test_data = image_generator_test.flow_from_directory(
    '/content/drive/MyDrive/Pistachio/test',
    target_size=(512, 512),
    color_mode="rgb",
    class_mode='categorical',
    batch_size=32
)

Found 431 images belonging to 2 classes.


In [12]:
history=model.fit(train_data,
                  epochs=50,
                  validation_data=test_data,
                  validation_steps=len(test_data),
                  verbose=1,
                  callbacks=callbacks)

Epoch 1/50
54/54 [==============================] - 989s 18s/step - loss: 55.7530 - accuracy: 0.5376 - precision: 0.5124 - recall: 0.7699 - val_loss: 0.6884 - val_accuracy: 0.5731 - val_precision: 0.5000 - val_recall: 1.0000
Epoch 2/50
54/54 [==============================] - 136s 3s/step - loss: 0.6893 - accuracy: 0.5527 - precision: 0.5311 - recall: 0.7065 - val_loss: 0.6862 - val_accuracy: 0.5731 - val_precision: 0.5731 - val_recall: 0.5731
Epoch 3/50
54/54 [==============================] - 134s 2s/step - loss: 0.6870 - accuracy: 0.5737 - precision: 0.5431 - recall: 0.6570 - val_loss: 0.6820 - val_accuracy: 0.5731 - val_precision: 0.5731 - val_recall: 0.5731
Epoch 4/50
54/54 [==============================] - 135s 3s/step - loss: 0.6864 - accuracy: 0.5591 - precision: 0.5412 - recall: 0.6960 - val_loss: 0.6853 - val_accuracy: 0.5731 - val_precision: 0.5731 - val_recall: 0.5731
Epoch 5/50
54/54 [==============================] - 133s 2s/step - loss: 0.6908 - accuracy: 0.5614 - preci